In [1]:
import main

In [2]:
import numpy as np

In [3]:
precursor_1 = main.precursor_FDM(delay_group_num=1)
precursor_1.set_beta([0.0075])
precursor_1.set_lambda([0.08])

In [4]:
material_1 = main.material_FDM(mat_id=0)
material_1.set_XS('XS_scatter',[[0,0.01],[0,0]])
material_1.set_XS('Diffusion_coef',[1.4,0.4])
material_1.set_XS('XS_absorption',[0.01,0.15])
# material_1.set_XS('XS_absorption',[0.01,0.10392197])
material_1.set_XS('XS_nu_fission',[0.007,0.2])
material_1.set_XS('XS_fission_spectrum',[1.0,0.0])

In [5]:
material_2 = main.material_FDM(mat_id=1)
material_2.set_XS('XS_scatter',[[0,0.01],[0,0]])
material_2.set_XS('Diffusion_coef',[1.4,0.4])
material_2.set_XS('XS_absorption',[0.01,0.15])
# material_2.set_XS('XS_absorption',[0.02,0.15])
material_2.set_XS('XS_nu_fission',[0.007,0.2])
material_2.set_XS('XS_fission_spectrum',[1.0,0.0])

In [6]:
material_3 = main.material_FDM(mat_id=2)
material_3.set_XS('XS_scatter',[[0,0.01],[0,0]])
material_3.set_XS('Diffusion_coef',[1.3,0.5])
material_3.set_XS('XS_absorption',[0.008,0.05])
# material_3.set_XS('XS_absorption',[0.018,0.05])
material_3.set_XS('XS_nu_fission',[0.003,0.06])
material_3.set_XS('XS_fission_spectrum',[1.0,0.0])

In [7]:
geo = main.geometry_FDM(x_block=3,y_block=3)
sl = slice(0, 3, None)
geo.set_block_mat(material_3,sl,sl)
geo.set_block_mat(material_1,1,1)
geo.set_block_mat(material_2,1,0)
geo.set_block_mat(material_2,0,1)

geo.set_block_size(x_size=[24,32,24],y_size=[24,32,24])
geo.set_discretized_num(x_dim=[24,32,24],y_dim=[24,32,24])
geo.check_blocks()

Material layout:
[[2 1 2]
 [1 0 2]
 [2 2 2]]
Block size:
x:[24, 32, 24]
y:[24, 32, 24]
Discretization number:
x:[24, 32, 24]
y:[24, 32, 24]


In [8]:
case = main.solver_FDM(folder_name='TWIGL_Seed_Blanket_2D_SCM_A1_100ms',group_num=2)
case.add_material([material_1,material_2,material_3])
case.set_geometry(geo,transient_mode=True)
case.set_precursor(precursor_1)
case.set_neutron_velocity([1e7,2e5])

In [9]:
# time_steps = np.array([0,0.005])
time_steps = np.linspace(0.0, 0.2, num=3)
# time_steps = np.array([0])
for time_index,time_step in enumerate(time_steps):
    print('Time={} (s)'.format(time_steps[time_index]))
    if time_index==0:
        # To get initial flux distribution
        case.solve_source_iter_correct(max_iter=50,k_tolerance=1e-5,flux_tolerance=1e-4,initial_k=1.0)
        # case.initial_dynamics(transient_algorithm='Implicit_Euler')
        case.initial_dynamics(time_steps=time_steps,transient_algorithm='SCM',vtk_save=False)
    else:
        # Update material
        if time_step <=0.2:
            m1_a2 = 0.15*(1-0.11667*time_step) # Linear 
        else:
            m1_a2 = 0.15*0.97666 # Jump
        material_1.set_XS('XS_absorption',[0.01,m1_a2])
        case.update_material(0,material_1,mat_type='real')
        # Solve transient problem with SCM
        time_interval = time_steps[time_index]-time_steps[time_index-1]
        case.solve_transient_SCM_2(time_index,time_interval,max_iter=100,k_tolerance=1e-6,flux_tolerance=1e-4,k_outer_tolerance=1e-7)
        # case.solve_transient_Implicit_Euler(time_index,time_interval,max_iter=200,k_tolerance=1e-5,flux_tolerance=1e-4)

Time=0.0 (s)


D:\Users\85136\anaconda3\lib\site-packages\scipy\sparse\linalg\dsolve\linsolve.py:318: SparseEfficiencyWarning: splu requires CSC matrix format
  warn('splu requires CSC matrix format', SparseEfficiencyWarning)


####################################
Timestep 0 calculation begins
Iteration 1: Eigenvalue k = 0.8446669513193322
Iteration 2: Eigenvalue k = 0.8899489163397002
Iteration 3: Eigenvalue k = 0.8994594776166441
Iteration 4: Eigenvalue k = 0.9038026479314235
Iteration 5: Eigenvalue k = 0.9064220494043876
Iteration 6: Eigenvalue k = 0.9081900679307713
Iteration 7: Eigenvalue k = 0.9094479494340285
Iteration 8: Eigenvalue k = 0.9103687896329571
Iteration 9: Eigenvalue k = 0.9110545506517275
Iteration 10: Eigenvalue k = 0.9115708863732314
Iteration 11: Eigenvalue k = 0.9119625156101641
Iteration 12: Eigenvalue k = 0.9122610517725338
Iteration 13: Eigenvalue k = 0.9124894190761463
Iteration 14: Eigenvalue k = 0.9126645386803547
Iteration 15: Eigenvalue k = 0.9127990586739625
Iteration 16: Eigenvalue k = 0.9129025182983143
Iteration 17: Eigenvalue k = 0.9129821584390484
Iteration 18: Eigenvalue k = 0.9130435007625038
Iteration 19: Eigenvalue k = 0.9130907698523009
Iteration 20: Eigenvalue k = 0

Iteration 4: Eigenvalue k = 0.994739402014911
Iteration 5: Eigenvalue k = 0.9947479037067389
Iteration 6: Eigenvalue k = 0.9947506549935241
Iteration 7: Met the criteria and k = 0.9947507915553577
Outer Iteration 3： kD=0.9947507915553577
Amplitude frequency:51.86016281472002
Amplitude frequency last:5.668818806338222
Amplitude frequency new:18.12808202411857
Total Q: 4.144371984046158
####################################
Outer iteration 4 begins
Iteration 1: Eigenvalue k = 0.9976172063532146
Iteration 2: Eigenvalue k = 0.9975448424580241
Iteration 3: Eigenvalue k = 0.9975095410471677
Iteration 4: Eigenvalue k = 0.9974930068702262
Iteration 5: Eigenvalue k = 0.9974856015755316
Iteration 6: Eigenvalue k = 0.9974826064012919
Iteration 7: Met the criteria and k = 0.9974817115629568
Outer Iteration 4： kD=0.9974817115629568
Amplitude frequency:18.12808202411857
Amplitude frequency last:51.86016281472002
Amplitude frequency new:-12.977592538300595
Total Q: 0.8749990636823022
#################